In [6]:
from transformers import SegformerImageProcessor, AutoModelForSemanticSegmentation
from PIL import Image
import requests
import matplotlib.pyplot as plt
import torch.nn as nn
import pandas as pd
import numpy as np
from optimum.onnxruntime import ORTModelForFeatureExtraction
from transformers import AutoTokenizer, PreTrainedTokenizerFast
import onnx
import onnxruntime
import tensorflow as tf
import torch

In [11]:
save_directory = "C:\\Users\\smyle\\Python\\StealMyLook\\notebook"
model_checkpoint = "mattmdjaga/segformer_b2_clothes"

In [12]:
# Load the model from hugging face
transformer_model = AutoModelForSemanticSegmentation.from_pretrained(model_checkpoint)
# Create an ONNX file from this model
transformer_model.save_pretrained(save_directory)


In [13]:
# Instantiate inference session
ort_session = onnxruntime.InferenceSession("model.onnx", providers=["CPUExecutionProvider"])
# Load a model from an ONNX file
ort_model = ORTModelForFeatureExtraction.from_pretrained(model_checkpoint, export=True)


Framework not specified. Using pt to export to ONNX.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.
Using the export variant default. Available variants are:
    - default: The default ONNX variant.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.
Using framework PyTorch: 2.2.0+cpu
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model'

In [14]:
# Load the processor from hugging face
processor = SegformerImageProcessor.from_pretrained("mattmdjaga/segformer_b2_clothes")

In [15]:
# Get image and preprocess it
Link = "http://assets.myntassets.com/v1/images/style/properties/7a5b82d1372a7a5c6de67ae7a314fd91_images.jpg"
image = Image.open(requests.get(Link, stream=True).raw)
inputs = processor(images=image, return_tensors="pt")

#Convert format for ONNX model
ortinputs = inputs["pixel_values"].numpy()
ort_inputs = {"pixel_values":ortinputs}

In [16]:
# outputs from transformer model and ONNX model
transformer_outputs = transformer_model(**inputs)
ort_outs = ort_session.run(None, ort_inputs)

In [17]:
# Get logits from transformer outputs
logits = transformer_outputs.logits.cpu()

# Get logits from ONNX outputs
logits = tf.nn.softmax(ort_outs,axis=-1)

In [18]:
#I cannot get this to run
upsampled_logits = nn.functional.interpolate(
    logits,
    size=image.size[::-1],
    mode="bilinear",
    align_corners=False,
)

pred_seg = upsampled_logits.argmax(dim=1)[0]
plt.imshow(pred_seg)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'dim'